## Code to check GPU integration


In [1]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for g in gpus:
            tf.config.experimental.set_memory_growth(g, True)
        print("Enabled memory growth for GPUs:", gpus)
    except Exception as e:
        print("Could not set memory growth:", e)
else:
    print("No GPUs found")


2025-11-09 18:23:31.178045: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-09 18:23:31.640466: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-09 18:23:33.501090: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Enabled memory growth for GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Imports


In [2]:
import pathlib, os
from pathlib import Path
import tensorflow as tf
from collections import Counter
import numpy as np
from tensorflow.keras import layers, models, optimizers, callbacks

## Constants

In [3]:
DATA_DIR = str("asl_alphabet_train")
IMG_SIZE = (200, 200)
BATCH_SIZE = 64
SEED = 42
VAL_SPLIT = 0.15

## Dataset imports

In [4]:
# training dataset (subset="training") and validation dataset (subset="validation")
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

Found 87000 files belonging to 29 classes.
Using 73950 files for training.


I0000 00:00:1762712637.168201   92642 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5561 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 87000 files belonging to 29 classes.
Using 13050 files for validation.


## Class checks

In [5]:
class_names = train_ds.class_names
print("Found classes:", len(class_names))
print(class_names)

Found classes: 29
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [7]:
from collections import Counter
p = pathlib.Path(DATA_DIR)
counts = {d.name: len(list(d.glob("*"))) for d in p.iterdir() if d.is_dir()}
for c, n in sorted(counts.items()):
    print(f"{c:20s} {n}")


A                    3000
B                    3000
C                    3000
D                    3000
E                    3000
F                    3000
G                    3000
H                    3000
I                    3000
J                    3000
K                    3000
L                    3000
M                    3000
N                    3000
O                    3000
P                    3000
Q                    3000
R                    3000
S                    3000
T                    3000
U                    3000
V                    3000
W                    3000
X                    3000
Y                    3000
Z                    3000
del                  3000
nothing              3000
space                3000


In [6]:
type(train_ds)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [7]:
iterator = iter(train_ds)

first_batch = next(iterator)

images_batch, labels_batch = first_batch


print(f"Type of images_batch: {type(images_batch)}")
print(f"Images batch shape: {images_batch.shape}")
print(f"Labels batch shape: {labels_batch.shape}")

print("\n--- Example from the batch ---")
print(f"Shape of one image: {images_batch[0].shape}")
print(f"Label for first image: {labels_batch[0]}")

Type of images_batch: <class 'tensorflow.python.framework.ops.EagerTensor'>
Images batch shape: (64, 200, 200, 3)
Labels batch shape: (64,)

--- Example from the batch ---
Shape of one image: (200, 200, 3)
Label for first image: 28


## Optimize pipeline

In [8]:
AUTOTUNE = tf.data.AUTOTUNE

# Basic preprocessing + augmentation layers
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.08),
    tf.keras.layers.RandomZoom(0.08),
])

# Normalization: scale 0-255 -> 0-1 and optionally standardize with ImageNet stats later in the model
preprocess_rescale = tf.keras.layers.Rescaling(1./255)

# Apply caching/prefetch and optionally augmentation only on training set
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.map(lambda x, y: (preprocess_rescale(x), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

val_ds = val_ds.map(lambda x, y: (preprocess_rescale(x), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


In [11]:
# Get model constants from your variables
num_classes = len(class_names)
input_shape = (*IMG_SIZE, 3) # (200, 200, 3)

model = models.Sequential([
    # Input Layer
    layers.Input(shape=input_shape),

    # --- Convolutional Block 1 ---
    # Learns basic edges and textures
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    # --- Convolutional Block 2 ---
    # Learns more complex shapes
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    # --- Convolutional Block 3 ---
    # Learns even more complex patterns
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    # --- Classifier Head ---
    # Flattens the 2D image data into a 1D vector
    layers.Flatten(),
    
    # A large dense layer to do the final learning
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5), # Dropout prevents overfitting
    
    # Output layer: one node for each class, softmax gives probabilities
    layers.Dense(num_classes, activation='softmax')
])

# Print a summary of the model
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 200, 200, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 200, 200, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 100, 100, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 100, 100, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 100, 100, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 50, 50, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 50, 50, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 50, 50, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 25, 25, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 80000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │    40,960,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 29)             │        14,877 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41,262,397 (157.40 MB)

 Trainable params: 41,262,397 (157.40 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # <-- Perfect for integer labels!
    metrics=['accuracy']
)

In [13]:
# Create a callback to stop training if validation accuracy doesn't improve
early_stopping = callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3, # Stop after 3 epochs of no improvement
    restore_best_weights=True # Automatically restore the best model weights
)

# Create a callback to save your best model to a file
model_checkpoint = callbacks.ModelCheckpoint(
    'best_asl_model.keras', # File name
    monitor='val_accuracy',
    save_best_only=True
)

In [14]:
EPOCHS = 20 # Start with 20, EarlyStopping will stop it if it's done sooner

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[early_stopping, model_checkpoint] # Pass in our helpers
)

Epoch 1/20


2025-11-09 19:14:22.666413: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-09 19:14:24.177305: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_893', 132 bytes spill stores, 132 bytes spill loads

2025-11-09 19:14:32.643041: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:382] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to o

1155/1156 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.1502 - loss: 2.9187

2025-11-09 19:17:22.570441: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-09 19:17:22.970544: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1030', 4 bytes spill stores, 4 bytes spill loads

2025-11-09 19:17:23.263799: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_893', 16 bytes spill stores, 16 bytes spill loads

2025-11-09 19:17:23.954345: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_893', 288 bytes

1156/1156 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.1504 - loss: 2.9181

...

1156/1156 ━━━━━━━━━━━━━━━━━━━━ 174s 150ms/step - accuracy: 0.9702 - loss: 0.0924 - val_accuracy: 0.9821 - val_loss: 0.0572
Epoch 19/20
   1/1156 ━━━━━━━━━━━━━━━━━━━━ 13:38 709ms/step - accuracy: 0.9531 - loss: 0.1336

2025-11-09 20:08:09.193897: W tensorflow/core/kernels/data/prefetch_autotuner.cc:55] Prefetch autotuner tried to allocate 30720256 bytes after encountering the first element of size 30720256 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


1156/1156 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.9685 - loss: 0.0977

2025-11-09 20:10:53.870285: W tensorflow/core/kernels/data/prefetch_autotuner.cc:55] Prefetch autotuner tried to allocate 30720256 bytes after encountering the first element of size 30720256 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


1156/1156 ━━━━━━━━━━━━━━━━━━━━ 174s 150ms/step - accuracy: 0.9700 - loss: 0.0935 - val_accuracy: 0.9966 - val_loss: 0.0099
